In [17]:
import numpy as np #提供ndarray操作之函式
import os #路徑操作的函式庫
import sklearn #機器學習演算法函式庫

path = '../input/flowers-recognition/flowers'

for flower_type in os.listdir(path): #查看要分類的目標有幾類
    print(flower_type)


In [18]:
classes = {'dandelion':0,'daisy':1,'sunflower':2,'tulip':3,'rose':4} #定義目標類別對應的編號

In [19]:
import cv2 #圖片讀入所需函式庫
import random #用他提供的函式來打散資料集

RESIZE_SIZE = 150 #統一輸入圖片大小
allImg = [] #希望存放所有改變尺寸後的圖片與對應的類別編號組成list的list
X = [] #放置其他特徵
Y = [] #放置目標特徵

def readImg(path): #從路徑讀入資料集
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            filename = os.path.join(dirname, filename)
            image = cv2.imread(filename ,cv2.IMREAD_COLOR) #以原始色彩讀入圖片
            resized_image = cv2.resize(image ,(RESIZE_SIZE,RESIZE_SIZE)) #縮放圖片尺寸
            #allImg.append([resized_image,classes.index(os.path.basename(dirname))])
            allImg.append([resized_image,classes[os.path.basename(dirname)]]) #在allImg加入每一張圖片及對應的類別組成的list
            #print(os.path.basename(dirname))
            #print(os.path.join(dirname, filename))
    random.shuffle(allImg) #打散串列元素
    for Img,f_type in allImg:
        X.append(Img)
        Y.append(f_type)
    return np.array(X),np.array(Y) #轉成ndarray以方便後續串列操作
            
X,Y = readImg(path) #讀入圖片並分離訓練特徵與目標特徵

print(X.dtype)
print(Y.dtype)

In [20]:
from sklearn.model_selection import train_test_split #用以將原始所提供資料集分為訓練集和測試集
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.1) #分割資料集為90%的訓練集及10%的測試集
#確認圖片形狀是否有成功重塑成150*150*3
print('X_train.shape={},Y_train.shape={}'.format(X_train.shape, Y_train.shape))
print('X_test.shape={},Y_test.shape={}'.format(X_test.shape, Y_test.shape))

In [21]:
#因為每一張圖片的所有像素值都是0~255，對每一像素除以255讓所有像素的值落在0~1之間
X_train = X_train / 255
X_test = X_test / 255

In [22]:
from keras.utils import np_utils #對目標特徵做one-hot-encoding，讓類別轉成0跟1以方便程式計算
Y_train = np_utils.to_categorical(Y_train)
Y_test_categories = Y_test
Y_test = np_utils.to_categorical(Y_test)
'''import tensorflow as tf
Y_train = tf.cast(Y_train,dtype = tf.int32)
Y_train = tf.squeeze(Y_train)
Y_train = tf.one_hot(Y_train,depth = 10)'''

In [23]:
from keras.models import Sequential #建立序列物件所需函式庫
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D #建立多層網路所需的個別網路所對應的函式庫

'''
Conv2D()參數 -> 
filters:核數，類似早期對一張圖用window去做卷積的概念(不太確定) 
kernel_size:核(window)的大小 ; 
padding:window卷積到圖片邊界的處理方式 
input_shape:輸入的圖片大小 
activation:激發函數，憑藉經驗去調整 
'''
#建立序列物件
model = Sequential() 
#建立卷積層搭配參數，輸出16個150*150*3的特徵圖
model.add(Conv2D(filters=16,kernel_size=(5,5),padding='same',input_shape=(150,150,3),activation='relu'))
#建立池化層搭配將上層輸出的特徵圖大小簡化成一半並保留重要特徵
model.add(MaxPooling2D(pool_size=(2,2)))
#建立卷積層並接收上層輸出後，搭配參數，輸出36個75*75*3的特徵圖
model.add(Conv2D(filters=36,kernel_size=(5,5),padding='same',input_shape=(75,75,3),activation='relu'))
#建立池化層搭配將上層輸出的特徵圖大小簡化成1/3並保留重要特徵
model.add(MaxPooling2D(pool_size=(3,3)))
#建立卷積層並接收上層輸出後，搭配參數，輸出56個25*25*3的特徵圖
model.add(Conv2D(filters=56,kernel_size=(5,5),padding='same',input_shape=(25,25,3),activation='relu'))
#建立池化層搭配將上層輸出的特徵圖大小簡化成1/3並保留重要特徵
model.add(MaxPooling2D(pool_size=(3,3)))
#隨機丟棄25%的神經元，讓每1次batch的訓練都可能訓練到不同的神經元，用以防止過擬合
model.add(Dropout(0.25))
#將上層的輸入平坦化，把所有特徵值轉為一維資料以供後續的全連結層使用
model.add(Flatten())
#隱藏層，用以提高模型準確度
model.add(Dense(256,activation='relu'))
#隨機丟棄50%的神經元，讓每1次batch的訓練都可能訓練到不同的神經元，用以防止過擬合
model.add(Dropout(0.5))
#輸出層，使用softmax作為輸出層的激發函數，第一個參數是要分類目標的數量
model.add(Dense(5,activation='softmax'))
#查看模型架構
model.summary()  


In [24]:
#定義訓練時的參數，損失函數用分類模型最常用的交叉熵，梯度下降法用最常用的adam，模型評估方式用準確度來評估
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
#訓練模型，將訓練集拆分80%為訓練集以及20%為訓練時的驗證集，將所有訓練集訓練10次(epochs=10)後終止訓練，每一批次(batch)訓練都是將300張圖片輸入，大概一個epoch會有3108/300個batch(3108是訓練集數量)
train_history = model.fit(x=X_train,y=Y_train,validation_data=(X_test, Y_test),validation_split=0.2,epochs=10,batch_size=300,verbose=2)

In [25]:
model.save('./model.h5') #儲存模型到本機

In [26]:
import matplotlib.pyplot as plt

#繪製訓練時的訓練集的準確度(或損失)和驗證集的準確度(或損失)
def show_train_history(train_history,standard,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title(standard)
    plt.ylabel('train')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='center right')
    plt.show()
    
show_train_history(train_history,'accuracy','accuracy','val_accuracy')
show_train_history(train_history,'loss','loss','val_loss')

In [27]:
from keras.models import load_model
model = load_model('../input/keras-cnn-model/model.h5') #先從本機上傳模型到input資料夾後，再載入訓練好的模型

In [28]:
scores = model.evaluate(X_test, Y_test)
scores[1] #只顯示模型的準確度(根據模型訓練的參數定義，可選其他指標值，比如recall)

In [29]:
prediction = model.predict(X_test) #對測試集預測

from sklearn.metrics import classification_report,confusion_matrix
#用以繪製confusion matrix之函式庫
import matplotlib.pyplot as plt
import seaborn as sns 
#一開始沒有下面的程式碼會報錯，查網路說因為預測向量與目標特徵向量內的元素不對稱，所以也要把預測向量做one-hot-encoding
for i in range(len(prediction)):
    max_value = max(prediction[i])
    for j in range(len(prediction[i])):
        if max_value == prediction[i][j]:
            prediction[i][j] = 1
        else:
            prediction[i][j] = 0
print(Y_test)
print(prediction)
print('分類報告:\n',classification_report(Y_test,prediction))
#使用參數average='macro'，期望輸出不受各類別樣本數影響
print('precision_score:',sklearn.metrics.precision_score(Y_test,prediction,average='macro'))
print('recall_score:',sklearn.metrics.recall_score(Y_test,prediction,average='macro'))
print('f1_score:',sklearn.metrics.f1_score(Y_test,prediction,average='macro'))
print('混淆矩陣:')
#因為confusion matrix不接受傳入one-hot-encoding的資料，所以要將測試資料以及預測向量做reverse-one-hot-encoding轉成10進位
plt.figure(figsize=(5,5))
sns.heatmap(confusion_matrix(Y_test.argmax(axis=1), prediction.argmax(axis=1)),annot=True,cmap=plt.cm.Blues)
plt.show()


In [30]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,20)) #設定整個繪製區域大小
rand = np.random.randint(0,len(X_test),10) #隨機從測試集取10個樣本來測試
axis = 1 #繪製子區域的座標起始值
#print(len(X_test))
#print(rand)
for i in rand:
    plt.subplot(5,2,axis) #row:5 col:2
    plt.imshow(X_test[i]) #顯示圖片
    plt.title("True : {} , Predict : {}".format(np.argmax(Y_test[i]) ,np.argmax(prediction[i]))) #對類別作one-hot-reverse-encoding轉成一開始所設定的類別編號
    axis+=1